In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df_train = pd.read_csv('Dataset/train.csv')
df_test = pd.read_csv('Dataset/test.csv')

In [3]:
df_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
df_test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [5]:
df_X_train = df_train.drop(columns=['Loan_Status']).copy()
df_X_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban


In [6]:
df_y_train = df_train[['Loan_ID', 'Loan_Status']].copy()
df_y_train.head()

,Loan_ID,Loan_Status
0,LP001002,Y
1,LP001003,N
2,LP001005,Y
3,LP001006,Y
4,LP001008,Y


In [7]:
df_X_test = df_test.copy()
df_X_test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


### EDA

In [8]:
def fillna_proportional(df, column):
    vcount = df[column].value_counts()
    proportion = vcount / vcount.sum()
    missing_count = df[column].isna().sum()
    to_fill_count = (proportion * missing_count).round().astype(int)

    while to_fill_count.sum() != missing_count:
        if to_fill_count.sum() < missing_count:
            to_fill_count[to_fill_count.idxmax()] += 1
        elif to_fill_count.sum() > missing_count:
            to_fill_count[to_fill_count.idxmin()] -= 1

    na_indices = df[df[column].isna()].index
    fill_values = np.concatenate([np.repeat(index, count)
                                 for index, count in to_fill_count.items()])
    np.random.shuffle(fill_values)
    df.loc[na_indices, column] = fill_values

    return df

In [9]:
def outlier_capping(data, upper_quantile=0.99, lower_quantile=0.01):
    upper_limit = data.quantile(upper_quantile)
    lower_limit = data.quantile(lower_quantile)
    # Cap the values using clip
    return data.clip(lower=lower_limit, upper=upper_limit)

In [24]:
from sklearn.preprocessing import StandardScaler

def preprocess(df, ID, treat_na=True, treat_outliers=False, upper_quantile=0.99, lower_quantile=0.01):
    # Step 1: Drop columns with excessive missing values
    missing_percentage = df.isnull().mean()
    columns_to_drop = missing_percentage[missing_percentage > 0.4].index.tolist()
    df = df.drop(columns=columns_to_drop)

    # Step 2: Identify numeric and categorical columns
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
    
    # Remove the ID column from categorical columns if it exists
    if ID in cat_cols:
        cat_cols.remove(ID)

    # Step 3: Normalize string categorical data - lowercase categorical columns
    df[cat_cols] = df[cat_cols].apply(lambda x: x.str.lower())

    # Step 4: Identify and map binary columns
    bin_cols = []
    for col in num_cols + cat_cols:
        if df[col].nunique() == 2:
            bin_cols.append(col)
            df[col] = df[col].map(lambda x: 1 if x == df[col].unique()[1] else (0 if x == df[col].unique()[0] else x))

    # Remove bin_cols from num_cols and cat_cols
    num_cols = [col for col in num_cols if col not in bin_cols]
    cat_cols = [col for col in cat_cols if col not in bin_cols]

    # print(f"Categorical Columns: {cat_cols}, \nBinary Columns: {bin_cols}, \nNumeric Columns: {num_cols}")

    # Step 5: Handle missing values if treat_na is True
    if treat_na:
        # Categorical and Binary Columns
        for col in (cat_cols + bin_cols):
            df = fillna_proportional(df, col)
        # Numerical Columns
        for num in num_cols:
            df[num] = df[num].fillna(df[num].median())

    # Step 6: Outlier treatment if treat_outliers is True
    if treat_outliers:
        for col in num_cols:
            df[col] = outlier_capping(df[col], upper_quantile, lower_quantile)
    
    # Step 7: Convert categorical columns to dummy variables
    df = pd.get_dummies(df ,columns=cat_cols, drop_first=True, dtype='int')

    # Step 8: Normalize Data
    scaler = StandardScaler()
    df[num_cols] = scaler.fit_transform(df[num_cols])

    return df, scaler


In [25]:
df_X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 57.7+ KB


In [26]:
df_X_train_cleaned, scaler = preprocess(df_X_train, ID='Loan_ID', treat_outliers=True)
df_X_train_cleaned.head()

,Loan_ID,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Dependents_2,Dependents_3+,Property_Area_semiurban,Property_Area_urban
0,LP001002,0.0,0.0,0,0.0,0.140201,-0.810858,-0.213994,0.274463,0.0,0,0,0,0,1
1,LP001003,0.0,1.0,0,0.0,-0.139573,0.013305,-0.213994,0.274463,0.0,1,0,0,0,0
2,LP001005,0.0,1.0,0,1.0,-0.489401,-0.810858,-1.004311,0.274463,0.0,0,0,0,0,1
3,LP001006,0.0,1.0,1,0.0,-0.581554,0.477853,-0.315970,0.274463,0.0,0,0,0,0,1
4,LP001008,0.0,0.0,0,0.0,0.173571,-0.810858,-0.048282,0.274463,0.0,0,0,0,0,1


In [28]:
df_X_test_cleaned, scaler = preprocess(df_X_test, ID='Loan_ID', treat_outliers=True)
df_X_test_cleaned.head()

,Loan_ID,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Dependents_2,Dependents_3+,Property_Area_semiurban,Property_Area_urban
0,LP001015,0.0,0,0,0.0,0.359840,-0.762204,-0.448487,0.268027,0.0,0,0,0,0,1
1,LP001022,0.0,0,0,0.0,-0.496150,-0.010730,-0.163351,0.268027,0.0,1,0,0,0,1
2,LP001031,0.0,0,0,0.0,0.126741,0.139565,1.297971,0.268027,0.0,0,1,0,0,1
3,LP001035,0.0,0,0,0.0,-0.734429,0.513299,-0.626697,0.268027,0.0,0,1,0,0,1
4,LP001051,0.0,1,1,0.0,-0.431401,-0.762204,-1.018759,0.268027,0.0,0,0,0,0,1


Recify - ` Married col contain null values in train df. no null in test`

In [30]:
df_X_train_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Loan_ID                  614 non-null    object 
 1   Gender                   614 non-null    float64
 2   Married                  614 non-null    float64
 3   Education                614 non-null    int64  
 4   Self_Employed            614 non-null    float64
 5   ApplicantIncome          614 non-null    float64
 6   CoapplicantIncome        614 non-null    float64
 7   LoanAmount               614 non-null    float64
 8   Loan_Amount_Term         614 non-null    float64
 9   Credit_History           614 non-null    float64
 10  Dependents_1             614 non-null    int64  
 11  Dependents_2             614 non-null    int64  
 12  Dependents_3+            614 non-null    int64  
 13  Property_Area_semiurban  614 non-null    int64  
 14  Property_Area_urban      6

In [31]:
df_X_train.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
dtype: int64

In [32]:
df_X_train_cleaned.isna().sum()

Loan_ID                    0
Gender                     0
Married                    0
Education                  0
Self_Employed              0
ApplicantIncome            0
CoapplicantIncome          0
LoanAmount                 0
Loan_Amount_Term           0
Credit_History             0
Dependents_1               0
Dependents_2               0
Dependents_3+              0
Property_Area_semiurban    0
Property_Area_urban        0
dtype: int64